## BMI-206 Course Individual Project
- Abolfazl (Abe) Arab

___
**Project plan**

In bioinformatics it’s common to have different tools for the same task. Generally, the main task we are working on is identifying cell-type specific casual enhancer-gene relationships inferred from the multi-modal mRNA and ATAC single-cell sequencing data. In addition to the SCENT approach as a peak-gene association inference method, there is a preprint that introduced the scMultiMap (Su et al., 2024) method – I found this method through searching the paper in which they cited SCENT paper. They claim to outperform in run-time (computational cost is less than 1% of SCENT and other existing methods) and statistical power. For instance, in systematic analyses of blood and brain data, scMultiMap shows appropriate type I error control, high statistical power with greater reproducibility across independent datasets and stronger consistency with orthogonal data modalities.

I will aim to apply it to the original single-cell datasets introduced in the SCENT paper. Then, I will evaluate the enhancers and target gene maps in disease-related cell types by using causal genetic variants in the genome-wide association studies (GWAS) as gold-labels. In our group project we aimed to perform more in-depth evaluations on the GWAS datasets and I will take advantage of our lessons there to extract a curated set of variants to evaluate scMultiMap outputs. Finally, I will implement a “causal variant enrichment analysis” formula through custom codes in R or python to reproduce similar analysis as described in the SCENT paper. As we move forward, we will decide what exact analysis we can perform. We have lots of examples from the sub-figures but we can also go beyond that, for instance we can evaluate what are the effects of changing covariates in the SCENT pipeline.

References:
- https://www.biorxiv.org/content/10.1101/2024.09.24.614814v1.full 
- https://changsubiostats.github.io/scMultiMap/articles/scMultiMap.html 

___

In [ ]:
# https://www.medrxiv.org/content/10.1101/2024.12.03.24318375v1.full-text
# !pip install KGWAS
# !conda install -y pytorch::pytorch
# import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '0' # '0' or '1' in lowell server :)
# from kgwas import KGWAS, KGWAS_Data

# data = KGWAS_Data(data_path = './data/kgwas/') ## initialize KGWAS data class with data path
# data.load_kg() ## load knowledge graph
# # data.load_external_gwas(PATH) ## load the GWAS file

# data.load_external_gwas(example_file = True)
# data.process_gwas_file()
# data.prepare_split()
# run = KGWAS(data, device = 'cuda:0', exp_name = 'test')
# run.initialize_model()
# run.load_pretrained('./data/kgwas/model/test')
# run.train(epoch = 10) ## train the model
# df_network_weight, df_variant_interpretation, disease_critical_network = run.get_disease_critical_network(variant_threshold = 5e-8, 
#                                                                                                             magma_path = None, magma_threshold = 0.05, program_threshold = 0.05,
#                                                                                                             K_neighbors = 3, num_cpus = 1)


In [ ]:
# !conda install -y bioconda::bioconductor-ensdb.hsapiens.v86
# !conda install -y bioconda::bioconductor-bsgenome.hsapiens.ucsc.hg38
# !conda install -y bioconda::bioconductor-biovizbase

In [8]:
%load_ext rpy2.ipython

In [10]:
%%R
library('scMultiMap')
library('SCENT')
library('Signac')
library('Seurat')
library(EnsDb.Hsapiens.v86)
library(BSgenome.Hsapiens.UCSC.hg38)

In [11]:
%%R
annotation <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)

  |                                                  | 0 % ~calculating   |++                                                | 4 % ~49s           |++++                                              | 8 % ~42s           |++++++                                            | 12% ~55s           |++++++++                                          | 16% ~54s           |++++++++++                                        | 20% ~50s           |++++++++++++                                      | 24% ~46s           |++++++++++++++                                    | 28% ~48s           |++++++++++++++++                                  | 32% ~44s           |++++++++++++++++++                                | 36% ~41s           |++++++++++++++++++++                              | 40% ~38s           |++++++++++++++++++++++                            | 44% ~37s           |++++++++++++++++++++++++                          | 48% ~33s           |++++++++++++++++++++++++++                        | 52% ~30s 

In addition: There were 24 warnings (use warnings() to see them)


In [13]:
%%R
annotation

GRanges object with 3021151 ranges and 5 metadata columns:
                  seqnames        ranges strand |           tx_id   gene_name
                     <Rle>     <IRanges>  <Rle> |     <character> <character>
  ENSE00001489430        X 276322-276394      + | ENST00000399012      PLCXD1
  ENSE00001536003        X 276324-276394      + | ENST00000484611      PLCXD1
  ENSE00002160563        X 276353-276394      + | ENST00000430923      PLCXD1
  ENSE00001750899        X 281055-281121      + | ENST00000445062      PLCXD1
  ENSE00001489388        X 281192-281684      + | ENST00000381657      PLCXD1
              ...      ...           ...    ... .             ...         ...
  ENST00000361739       MT     7586-8269      + | ENST00000361739      MT-CO2
  ENST00000361789       MT   14747-15887      + | ENST00000361789      MT-CYB
  ENST00000361851       MT     8366-8572      + | ENST00000361851     MT-ATP8
  ENST00000361899       MT     8527-9207      + | ENST00000361899     MT-ATP6
  ENS

In [14]:
%%R
readRDS('data/sc-datasets/SCENT_obj_all.rds') -> SCENT_obj_all

In [3]:
# %%R
# readRDS('data/sc-datasets/atac_matrix.rds') -> atac_matrix
# readRDS('data/sc-datasets/rna_matrix.rds') -> rna_matrix

In [15]:
%%R
CreateSeuratObject(counts = SCENT_obj_all@rna, assay = "RNA", project = "BMI206") -> rna

In [16]:
%%R
rna

An object of class Seurat 
36601 features across 31547 samples within 1 assay 
Active assay: RNA (36601 features, 0 variable features)
 1 layer present: counts


In [17]:
%%R
CreateSeuratObject(counts = SCENT_obj_all@atac, assay = "ATAC", project = "BMI206") -> atac

In [19]:
%%R
atac

An object of class Seurat 
132520 features across 31547 samples within 1 assay 
Active assay: ATAC (132520 features, 0 variable features)
 1 layer present: counts


___

In [ ]:
%%R
data <- rna

data[['peak']] <- CreateChromatinAssay(
  counts = SCENT_obj_all@atac,
  sep = c(":", "-"),
  fragments = fragpath,
  annotation = annotation
)

In [24]:
%%R
data@assays$peak['counts'][1:5, 1:5]

5 x 5 sparse Matrix of class "dgCMatrix"
                   BRI-1281_AAACAGCCAATTTGGT BRI-1281_AAACAGCCAGCACCAT
chr1-817238-817438                         .                         .
chr1-827432-827632                         .                         .
chr1-869776-869976                         .                         .
chr1-904365-904565                         .                         .
chr1-904661-904861                         .                         .
                   BRI-1281_AAACATGCAGGCTAGA BRI-1281_AAACATGCATGCTTAG
chr1-817238-817438                         .                         .
chr1-827432-827632                         .                         .
chr1-869776-869976                         .                         .
chr1-904365-904565                         .                         .
chr1-904661-904861                         .                         .
                   BRI-1281_AAACATGCATTGCAGC
chr1-817238-817438                         .
chr1-827432-82763

In [25]:
%%R
data$peak

ChromatinAssay data with 132520 features for 31547 cells
Variable features: 0 
Genome: 
Annotation present: TRUE 
Motifs present: FALSE 
Fragment files: 0 


In [29]:
%%R
pairs_df <- get_top_peak_gene_pairs(data,
                                    gene_top=20000, 
                                    peak_top=60000,
                                    distance = 5e+7,
                                    gene_assay = 'RNA', peak_assay = 'peak')

In [30]:
%%R
pairs_df

[1] gene peak
<0 rows> (or 0-length row.names)


I'm not sure why `pairs_df` is empty. I will try to debug it!